# Regression with a Tabular Gemstone Price Dataset
## Aplicando regressão linear para predição de valores de pedras preciosas.

### Baixando Dataset Kaggle

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
dataset_name = 'competitions/playground-series-s3e8'
od.download_kaggle_dataset(dataset_name, 'data')

### Lendo Dataset e Tratando Dados

In [4]:
import pandas as pd

train_df = pd.read_csv('data/playground-series-s3e8/train.csv', index_col=0)
train_df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


É possível verificar que existem algumas entradas de texto a serem tratadas no Dataset.
A primeira é a cor, que serve apenas para categorizar sem estabelecer uma ordem entre os dados (variáveis nominais).
As outras são a clareza e o corte, categorias que estabelecem uma ordem de precedência, por exemplo, "IF" está acima de "VVS1" (variáveis ordinal).
As variáveis nominais podem ser tratadas utilizado no "one-hot encoding", conforme abaixo.

In [5]:
train_df = pd.get_dummies(train_df, columns=['color'])
train_df.head()

,carat,cut,clarity,depth,table,x,y,z,price,color_D,color_E,color_F,color_G,color_H,color_I,color_J
id,,,,,,,,,,,,,,,,
0,1.52,Premium,VS2,62.2,58.0,7.27,7.33,4.55,13619,0,0,1,0,0,0,0
1,2.03,Very Good,SI2,62.0,58.0,8.06,8.12,5.05,13387,0,0,0,0,0,0,1
2,0.70,Ideal,VS1,61.2,57.0,5.69,5.73,3.50,2772,0,0,0,1,0,0,0
3,0.32,Ideal,VS1,61.6,56.0,4.38,4.41,2.71,666,0,0,0,1,0,0,0
4,1.70,Premium,VS2,62.6,59.0,7.65,7.61,4.77,14453,0,0,0,1,0,0,0


O one-hot encoding funciona criando uma coluna para cada valor único dentre os valores da coluna especificada, sendo seu valor 0 ou 1.
Para as variáveis ordinais é possível simplesmente atribuir um valor numérico contínuo.

In [6]:
train_df['cut'] = train_df['cut'].astype('category')
train_df['cut'] = train_df['cut'].cat.reorder_categories(
    ['Premium', 'Ideal', 'Very Good', 'Good', 'Fair'],
    ordered=True
)
train_df['cut'] = train_df['cut'].cat.codes

In [7]:
train_df['clarity'] = train_df['clarity'].astype('category')
train_df['clarity'] = train_df['clarity'].cat.reorder_categories(
    ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1'],
    ordered=True
)
train_df['clarity'] = train_df['clarity'].cat.codes

In [8]:
train_df.head()

,carat,cut,clarity,depth,table,x,y,z,price,color_D,color_E,color_F,color_G,color_H,color_I,color_J
id,,,,,,,,,,,,,,,,
0,1.52,0,4,62.2,58.0,7.27,7.33,4.55,13619,0,0,1,0,0,0,0
1,2.03,2,6,62.0,58.0,8.06,8.12,5.05,13387,0,0,0,0,0,0,1
2,0.70,1,3,61.2,57.0,5.69,5.73,3.50,2772,0,0,0,1,0,0,0
3,0.32,1,3,61.6,56.0,4.38,4.41,2.71,666,0,0,0,1,0,0,0
4,1.70,0,4,62.6,59.0,7.65,7.61,4.77,14453,0,0,0,1,0,0,0


### Preparando Dados e Gerando Modelo de Regressão

Aqui separamos a coluna do target das colunas das features. 

In [9]:
y = train_df['price']
X = train_df[train_df.columns.drop('price')]

Separamos os Datasets acima em 75% para treino e o restante para testarmos a predição.

In [10]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.25, random_state=42
)

Por fim, importamos o modelo de regressão linear do SciKit-Learn e aplicamos os Datasets de treino. Buscamos o score do modelo gerado utilizando os Datasets de treino.

In [11]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(train_X, train_y)

Obtendo algumas métricas de precisão do modelo.

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = reg.predict(test_X)

# Coeficiente de Determinação (R2)
r2 = reg.score(X, y)

# Erro Médio Absoluto (MAE)
mae = mean_absolute_error(test_y, y_pred)

# Erro Médio Quadrado
mse = mean_squared_error(test_y, y_pred)

print("Coeficiente de Determinação:", r2)
print("Erro Médio Absoluto:", mae)
print("Erro Médio Quadrado:", mse)


Coeficiente de Determinação: 0.9395181962956224
Erro Médio Absoluto: 658.1110198434593
Erro Médio Quadrado: 992413.736030248
